# Hackathon Query Notebook

Welcome to the hackathon! This notebook is your starting point for querying the main dataset.

It's designed to connect to a **remote Tentris server** that is already running and loaded with all the data. You do not need to run or install Tentris yourself.

**Your only task:** Follow the setup steps, run the cells, and start writing your own queries!

### Step 1: Setup env

Follow the instructions in the [README.md](https://github.com/sdsc-ordes/open-pulse-quickstart/blob/feat/tentris-quickstart/src/tentris-quickstart/README.md)

### Step 2: Imports and Configuration

This cell imports the libraries and sets up our connection variables.

In [ ]:
import rdflib
import pandas as pd
import tentris 
from tentris import TentrisHTTPStore
import urllib.error
from IPython.display import display, Markdown
import os
from dotenv import load_dotenv
import requests

load_dotenv()

#Configuration
ENDPOINT_URL = "http://128.178.219.51:7502"
USER = os.getenv("TENTRIS_USER") 
PASS = os.getenv("TENTRIS_PASSWORD")


if USER and PASS:
    display(Markdown("✅ Environment loaded successfully."))
else:
    display(Markdown("⚠️ Please set `TENTRIS_USER` and `TENTRIS_PASSWORD` in your `.env` file."))

def login(url: str, user: str, password: str) -> str:
    s = requests.Session()
    s.post(f"{url}/login", {"username": user, "password": password})
    c = s.cookies.get("tentris")
    
    return f"tentris={c}"

graph = None

✅ Environment loaded successfully.

### Step 3: Connect to the Server

This cell creates the `rdflib.Graph` object. It uses the `TentrisHTTPStore`, which is optimized to work with the Tentris server.

**Note:** This cell also runs a test query (`ASK { ?s ?p ?o }`) to make sure the server is reachable. If this cell fails, please double-check the `ENDPOINT_URL` you set in Step 2.

In [22]:
# --- Connect to Tentris Server ---

def login(url: str, user: str, password: str) -> str:
    s = requests.Session()
    resp = s.post(f"{url}/login", {"username": user, "password": password})
    resp.raise_for_status()
    cookie = s.cookies.get("tentris")
    return f"tentris={cookie}"

display(Markdown(f"🚀 Connecting to `{ENDPOINT_URL}`..."))

cookie = login(ENDPOINT_URL, USER, PASS)
display(Markdown(f"✅ Logged in as `{USER}`"))

graph = rdflib.Graph(store=TentrisHTTPStore(ENDPOINT_URL, headers={"Cookie": cookie}))

results = graph.query("SELECT * WHERE { ?s ?p ?o } LIMIT 10")
for row in results:
    print(row)



🚀 Connecting to `http://128.178.219.51:7502`...

✅ Logged in as `openpulse`

(rdflib.term.URIRef('http://example.org/software/81250'), rdflib.term.URIRef('http://schema.org/contentUrl'), rdflib.term.URIRef('http://example.org/downloads/software81250.zip'))
(rdflib.term.URIRef('http://example.org/software/81250'), rdflib.term.URIRef('http://schema.org/datePublished'), rdflib.term.Literal('2000-11-23', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date')))
(rdflib.term.URIRef('http://example.org/software/81250'), rdflib.term.URIRef('http://schema.org/url'), rdflib.term.URIRef('http://example.org/software/81250/'))
(rdflib.term.URIRef('http://example.org/software/81250'), rdflib.term.URIRef('http://schema.org/dateCreated'), rdflib.term.Literal('2000-11-23', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date')))
(rdflib.term.URIRef('http://example.org/software/81250'), rdflib.term.URIRef('http://schema.org/license'), rdflib.term.URIRef('https://spdx.org/licenses/MIT.html'))
(rdflib.term.URIRef('http://example.org/software/81250'), rd

### Step 4: Run Example Queries

Now you're ready to go! The `graph` object is your gateway to the database.

Here are a few examples to get you started. You can (and should!) modify these and create new cells to write your own.

Use the helper function run_query() to easily run a single or multiline SPARQL query.

In [30]:
# Example 1: Count all triples in the database

def run_query(query: str, description: str = ""):
    if description:
        display(Markdown(f"**Running query:** {description}"))
    results = graph.query(query)
    df = pd.DataFrame([r.asdict() for r in results])
    display(df)

# --- Example Queries ---

run_query("SELECT (COUNT(*) AS ?totalTriples) WHERE { ?s ?p ?o }", "Counting all triples")
run_query("SELECT * WHERE { ?s ?p ?o } LIMIT 10", "Showing 10 example triples")


**Running query:** Counting all triples

,totalTriples
0,2000000


**Running query:** Showing 10 example triples

,s,p,o
0,http://example.org/software/81250,http://schema.org/contentUrl,http://example.org/downloads/software81250.zip
1,http://example.org/software/81250,http://schema.org/datePublished,2000-11-23
2,http://example.org/software/81250,http://schema.org/url,http://example.org/software/81250/
3,http://example.org/software/81250,http://schema.org/dateCreated,2000-11-23
4,http://example.org/software/81250,http://schema.org/license,https://spdx.org/licenses/MIT.html
5,http://example.org/software/81250,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://schema.org/SoftwareSourceCode
6,http://example.org/software/81250,http://schema.org/name,Example Software 81250
7,http://example.org/software/81250,https://open-pulse.epfl.ch/ontology#repository...,https://open-pulse.epfl.ch/ontology#Software
8,http://example.org/software/81250,http://schema.org/codeRepository,http://github.com/exampleorg/repo81250
9,http://example.org/software/81250,http://schema.org/programmingLanguage,Rust


### Step 5: Your Turn! (Happy Hacking)

This is your canvas. Create new code cells below this one and start building your project.

**Tip:** Don't forget to add `LIMIT 10` to your queries while you are exploring, so you don't accidentally try to print a million rows!

In [29]:
run_query("""
PREFIX schema: <http://schema.org/>
PREFIX pulse: <https://open-pulse.epfl.ch/ontology#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?repoName ?disciplineIRI ?disciplineDescription
WHERE {
  ?repo a schema:SoftwareSourceCode ;
        schema:name ?repoName ;
        pulse:discipline ?disciplineIRI .
  SERVICE <https://query.wikidata.org/sparql> {
    ?disciplineIRI schema:description ?disciplineDescription .
    FILTER(LANG(?disciplineDescription) = "en")
  }
}
LIMIT 100
""", "Federated query with Wikidata")


**Running query:** Federated query with Wikidata

,repoName,disciplineIRI,disciplineDescription
0,Example Software 67290,http://www.wikidata.org/entity/Q83588,branch of science that applies physical scienc...
1,Example Software 78180,http://www.wikidata.org/entity/Q83588,branch of science that applies physical scienc...
2,Example Software 5235,http://www.wikidata.org/entity/Q83588,branch of science that applies physical scienc...
3,Example Software 80745,http://www.wikidata.org/entity/Q83588,branch of science that applies physical scienc...
4,Example Software 32640,http://www.wikidata.org/entity/Q83588,branch of science that applies physical scienc...
...,...,...,...
95,Example Software 95955,http://www.wikidata.org/entity/Q83588,branch of science that applies physical scienc...
96,Example Software 58650,http://www.wikidata.org/entity/Q83588,branch of science that applies physical scienc...
97,Example Software 82050,http://www.wikidata.org/entity/Q83588,branch of science that applies physical scienc...
98,Example Software 95595,http://www.wikidata.org/entity/Q83588,branch of science that applies physical scienc...


In [27]:
run_query(""" PREFIX schema: <http://schema.org/> 
    PREFIX pulse: <https://open-pulse.epfl.ch/ontology#> 
    PREFIX wd: <http://www.wikidata.org/entity/>

SELECT ?personName (COUNT(DISTINCT ?repo) AS ?repoCount)
WHERE {
  # 1. Find repositories matching the criteria
  ?repo a schema:SoftwareSourceCode ;
        schema:programmingLanguage ?language ;
        pulse:discipline wd:Q2329 . # wd:Q2329 is the IRI for Chemistry
  
  # 2. Filter for Python (case-insensitive)
#   FILTER(CONTAINS(LCASE(STR(?language)), "python"))
  
  # 3. Find the author of these repos
  ?repo schema:author ?person .
  
  # 4. Ensure the author is a Person and get their name
  ?person a schema:Person ;
          schema:name ?personName .
}
GROUP BY ?person ?personName
ORDER BY DESC(?repoCount)
LIMIT 10
""", "Top 10 authors for Python repos in Chemistry")



**Running query:** Top 10 authors for Python repos in Chemistry

,personName,repoCount
0,Person 72404,1
1,Person 86129,1
2,Person 97874,1
3,Person 74654,1
4,Person 60704,1
5,Person 52829,1
6,Person 96344,1
7,Person 9764,1
8,Person 71054,1
9,Person 18134,1
